In [75]:
import numpy as np
import pandas as pd
from src.utils import * 

In [11]:
gbsg = pd.read_csv("../GBSG2")

# On a un jeu de données complet: 
np.where(gbsg.isna()==True)

(array([], dtype=int64), array([], dtype=int64))

On détermine $I$ et $J$: 

In [12]:
I, J = gbsg.shape

Trouver $K_1$, $K_2$

In [16]:
# Variables continues
idx_k1 = gbsg.columns[(gbsg.isin([0, 1])==False).any()]

#Variables encodées en dummy variables 
idx_j = gbsg.columns[(gbsg.isin([0, 1])).all()]

In [46]:
s = df1.std(axis=0).to_numpy()
p = np.sqrt(df2.sum(axis=0)/df2.shape[0]).to_numpy() 

array([0.75977594, 0.59883268, 0.66019698])

In [47]:
class IterativeFAMDImputer(FAMD):
    def __init__(self, n_components=2):
        """Initialisation
        Args:
            n_components (int, optional): _description_. Defaults to 2.
            data (_type_, optional): _description_. Defaults to None.
        """
        self.n_components = n_components

    def inital_impute(self, data):
        #Pour les variables continues
        Ximp_C0 = data[self.k1].copy()  
        mean = data[self.k1].mean(axis=0)
        Ximp_C0.fillna(mean) # imputation par la moyenne
        self.df_C0 = Ximp_C0

        # Pour les variables catégorielles 
        Ximp_categ = data[self.j].copy()  
        pj = data[self.j].sum(axis=0)/data.shape[0]
        Ximp_categ.fillna(pj) # imputation par la proportion pj
        self.df_categ = Ximp_categ
        pass 
        
    def impute(self, Xobs, n_it, tol=1e-4,verbose=False):
        
        # Initialisation 
        idx_NA = Xobs.isna().astype(int)
        #Initial imputation 
        self.inital_impute(Xobs)
        self.df() # on construit les données avec df_C0 et df_categ remplis 

        self.DM()
        Ximp = self.XD_moins_sqrt - self.M 
        diff = np.inf
        i= 0
        while i < n_it and diff > tol: 
            U, S, Vt = np.linalg.svd(Ximp.values)
            Z_p = pd.DataFrame(U[:,:self.n_components]@ np.diag(S)[:self.n_components,:self.n_components] @Vt[:self.n_components,:], columns=Ximp.columns, index=Ximp.index)
            Ximp_ = (1-M)*Z_p + M*Ximp
            mu += Ximp_.mean(axis=0)
            Ximp = Ximp_ - Ximp_.mean(axis=0)
            Xreconstr = Ximp + mu 
            diff = ((Xreconstr - Ximp - Ximp_.mean(axis=0) )**2).mean().mean()
            i +=1 
        if i < n_it: 
            print('Converged in', i)
        else: 
              print('Maximum iterations reached')    
        return Xreconstr